In [3]:
from copy import copy

In [4]:
VAR_INDEX = 0

# Abstract class Formula
class Formula(): 
    def __init__(self, args, negated=False):
        self.negated = negated
        self.args = args  
    def __neg__(self): 
        new_object = copy(self)
        new_object.negated = not new_object.negated
        return new_object 
    def __add__(self, other): return Disjunction(self, other)
    def __mul__(self, other): return Conjunction(self, other)
    def __rshift__(self, other): return Implication(self, other)
    def __mod__(self, other): return Equivalence(self, other)
        
    
    

# Abstract class Connective 
class Connective(Formula): 
    def __init__(self, left, right):
        super().__init__([left, right], False)

# Conjunction
class Conjunction(Connective):
    pass

# Disjunction
class Disjunction(Connective): 
    pass

# Implication
class Implication(Formula):
    def __init__(self, left, right):
        super().__init__([left, right], False)

# Equivalence
class Equivalence(Formula): 
    def __init__(self, left, right):
        super().__init__([left, right], False)

# Variable
class Var(Formula):
    def __init__(self, name, specific_negated=False):
        super().__init__(None, negated=specific_negated)        
        global VAR_INDEX
        self.index = VAR_INDEX
        VAR_INDEX += 1
        self.name = name

In [5]:
def remove_imp_eq(formula):
    if type(formula) == Var:
        return formula
    
    if type(formula) == Conjunction:
        expr = remove_imp_eq(formula.args[0]) * remove_imp_eq(formula.args[1])
        expr.negated = formula.negated
        return expr
    
    if type(formula) == Disjunction:
        expr = remove_imp_eq(formula.args[0]) + remove_imp_eq(formula.args[1])
        expr.negated = formula.negated
        return expr
    
    if type(formula) == Implication:
        expr = remove_imp_eq(-(formula.args[0])) + remove_imp_eq(formula.args[1])
        expr.negated = formula.negated
        return expr
    
    if type(formula) == Equivalence:
        expr = remove_imp_eq(formula.args[0]) * remove_imp_eq(formula.args[1]) + \
               remove_imp_eq(-(formula.args[0])) * remove_imp_eq(-(formula.args[1]))
        expr.negated = formula.negated
        return expr

In [375]:
def de_morgan(formula):
    if type(formula) == Var:
        return formula
    if type(formula) == Disjunction :
        if formula.negated:
            return de_morgan(-formula.args[0]) * de_morgan(-formula.args[1])
        else:
            return de_morgan(formula.args[0]) + de_morgan(formula.args[1])
    if type(formula) == Conjunction:
        if formula.negated:
            return de_morgan(-formula.args[0]) + de_morgan(-formula.args[1])
        else:
            return de_morgan(formula.args[0]) * de_morgan(formula.args[1])
    
        
a = Var('a')
b = Var('b')
c = Var('c')
d = Var('d')

f = -(a * b) + --(c * d) + a + --(a + a)
d = de_morgan(f)

In [377]:
d.args[0].args[0].args[1]

In [277]:
def remove_associativity(formula):
    if type(formula) == Var:
        return formula
#     if type(formula) == Disjunction:
    if type(formula.args[0]) == Disjunction:
        return [remove_associativity(formula.args[0]), formula.args[1]]
    else:
        print(formula.args)
        return [formula.args[0], formula.args[1]]       
        
def flatten_list(x):
    if type(x[0]) == list:
        return flatten_list(x[0]) + [x[1]]
    return x
            

In [323]:
a = Var('a')
b = Var('b')
c = Var('c')
d = Var('d')

f = -(a * b) + -(c * d) + a + -(a + a)

In [324]:
de_morgan(f)

In [319]:
e.negated

True

In [306]:
remove_associativity(f)

[<__main__.Conjunction object at 0x000002B69292E0B8>, <__main__.Conjunction object at 0x000002B69292E748>]


[[[<__main__.Conjunction at 0x2b69292e0b8>,

In [268]:
a, b, c, d

(<__main__.Var at 0x2b692453f60>,
 <__main__.Var at 0x2b692453cc0>)

In [707]:
f = (a % a) >> (b % b)
r = remove_imp_eq(f)
print(r.args[0].name, r.args[0].negated)
print(r.args[1].name, r.args[1].negated)
print(type(r))

impl
eq
eq


AttributeError: 'Disjunction' object has no attribute 'name'